![University of Tehran](./img/UT.png)
#   <font color='red'><center>AI CA 2 - GA<center></font> 
## <center>Dr. Fadaei<center>
### <center>Daniyal Maroufi<center>
### <center>810098039<center>

## Aim

This assignment aims to use the Genetic Algorithm (GA) to find an encoded text's key string (14 characters). We prepared a global text to generate the dictionary, and we will use it to decode the text.

## Problem Description



# Algorithm Description

First, we generate 300 possible answers (chromosomes) as the initial population or the first generation. Then, we use a fitness function to rank the chromosomes. The fitness function presents how good each chromosome (solution) is. By comparing all chromosomes, we can have the best chromosomes of the generations. Then we will create a new generation based on these selected chromosomes using crossover and mutation processes. By repeating this process, we will finally find the solution that satisfies the expectations of the solution.



In [2]:
class Decoder():

    def __init__(self, globalText, encodedText, keyLength=14):
        self.globalText=globalText
        self.encodedText=encodedText
        self.keyLength=keyLength

    def generateInitialPopulation(self):
        pass

    def crossOver(self, chromosome1, chromosome2):
        pass

    def mutate(self, chromosome):
        pass

    def calculateFitness(self, chromosome):
        pass

    def rankPopulation(self):
        pass

    def generateNewPopulation(self):
        pass

    def decode(self):
        pass
